In [1]:
!pip install git+https://github.com/sjgallagher2/PyWORDS
!pip install cltk
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
import unicodedata
import pywords.lookup as lookup
#from urllib.request import urlopen
#pip install html5lib
#pip install lxml

  Cloning https://github.com/sjgallagher2/PyWORDS to /tmp/pip-req-build-25pc4kvf
  Running command git clone --filter=blob:none --quiet https://github.com/sjgallagher2/PyWORDS /tmp/pip-req-build-25pc4kvf
  Resolved https://github.com/sjgallagher2/PyWORDS to commit 815f1fad4c560e89c6200df1e0833e2cdf062338
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pywords.lookup as lookup

word = 'lacedaemon'
# High level
lookup.lookup_word(word)  # Print matching words

# Lower level
# matches = lookup.match_word(word)
# for match in matches:
#      print(lookup.get_dictionary_string(match,full_info=True))


In [3]:
for match in lookup.match_word('graecum'): # Match possible words
    print(lookup.get_dictionary_string(match, False, True, False).split(' ')[0].replace(',',""))  # Start with the definition of the word


graecus


In [4]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
PATH= 'drive/Shareddrives/AI in Classics/GreekABASAModel/DataSetGenerator'

os.chdir(PATH)# sets the current active directory to the current folder
!pwd
# IF YOU ENCOUNTER AN ERROR FIRST TRY RESTARTING RUNTIME

Mounted at /content/drive
/content/drive/Shareddrives/AI in Classics/GreekABASAModel/DataSetGenerator


In [1]:
!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

#test2 #allot easier proably just going to use this
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()


def generate_variations(word,language='greek',show=False):
      # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
      #DECLINES WORDS
      #test words
      #'Ῥωμαῖος' romans
      #'μισώ' hate
      #'εἰμί' are to be
      #word = word.lower()

# takes in a word as a string and uses the collatinus decliner to return
# the different forms of the word. returns words which is a list that contains
# tuples formated like so: [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
  def decline(word):
    decliner = CollatinusDecliner()
    words = decliner.decline(word)
    return words

  # takes in a declined word object (list of words in the format returned by the decliner function) and
  # formats it so that each word is with the correct type (nominative, genative, etc.)
  # currently only works with nouns that have one gender. needs to be expanded so it can make
  # dictionaries for multiple genders and word types.
  def fill_dicts(word, neutral = False, masc = False, fem = False):
      singular_dict = {}
      plural_dict = {}
      for i in range(0, len(word)):

        #check if verb
        if word[i][1][0] == "v":

          #check if singular verb
          if word[i][1][1:] == "1spia---":
            singular_dict["First Person"] = (word[i][0])
          if word[i][1][1:] == "2spia---":
            singular_dict["Second Person"] = (word[i][0])
          if word[i][1][1:] == "3spia---":
            singular_dict["Third Person"] = (word[i][0])
          #check if plural verb
          if word[i][1][1:] == "1ppia---":
            plural_dict["First Person"] = (word[i][0])
          if word[i][1][1:] == "2ppia---":
            plural_dict["Second Person"] = (word[i][0])
          if word[i][1][1:] == "3ppia---":
            plural_dict["Third Person"] = (word[i][0])

        # check if singular
        elif word[i][1][0] == "-":
          if word[i][1][2] == "s":
            if word[i][1][7] == "n":
              singular_dict["Nominative"] = (word[i][0])
            elif word[i][1][7] == "g":
              singular_dict["Genative"] = (word[i][0])
            elif word[i][1][7] == "d":
              singular_dict["Dative"] = (word[i][0])
            elif word[i][1][7] == "a":
              singular_dict["Accusative"] = (word[i][0])
            elif word[i][1][7] == "b":
              singular_dict["Ablative"] = (word[i][0])
            elif word[i][1][7] == "v":
              singular_dict["Vocative"] = (word[i][0])
          # check if plural
          elif word[i][1][2] == "p":
            if word[i][1][7] == "n":
              plural_dict["Nominative"] = (word[i][0])
            elif word[i][1][7] == "g":
              plural_dict["Genative"] = (word[i][0])
            elif word[i][1][7] == "d":
              plural_dict["Dative"] = (word[i][0])
            elif word[i][1][7] == "a":
              plural_dict["Accusative"] = (word[i][0])
            elif word[i][1][7] == "b":
              plural_dict["Ablative"] = (word[i][0])
            elif word[i][1][7] == "v":
              plural_dict["Vocative"] = (word[i][0])
      return singular_dict, plural_dict




  def make_df(word_tup, noun = False):
    print(word_tup)

    #gender = set()
    #for i in range(0,len(word)):
    #  gender.add(word[i][1][6])
    # runs if the word has no gendered attributes
    #if '-' in gender or 'n' in gender:
    singular_dict, plural_dict = fill_dicts(word_tup, True)
    print(singular_dict, plural_dict)
    print()

    if word_tup[1][1][0] == "v":
        dict_verb = {"Person": ["First Person", "Second Person", "Third Person"], "Singular":[], "Plural":[]}
        dict_verb["Singular"].append(singular_dict["First Person"])
        dict_verb["Singular"].append(singular_dict["Second Person"])
        dict_verb["Singular"].append(singular_dict["Third Person"])
        dict_verb["Plural"].append(singular_dict["First Person"])
        dict_verb["Plural"].append(singular_dict["Second Person"])
        dict_verb["Plural"].append(singular_dict["Third Person"])
        df = pd.DataFrame(dict_verb)
        df = df.set_index("Person")
        return df

    if word_tup[1][1][0] == "-":
      dict_overall = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
      dict_overall["Singular"].append(singular_dict["Nominative"])
      dict_overall["Singular"].append(singular_dict["Genative"])
      dict_overall["Singular"].append(singular_dict["Dative"])
      dict_overall["Singular"].append(singular_dict["Accusative"])
      dict_overall["Singular"].append(singular_dict["Ablative"])
      dict_overall["Singular"].append(singular_dict["Vocative"])

      dict_overall["Plural"].append(plural_dict["Nominative"])
      dict_overall["Plural"].append(plural_dict["Genative"])
      dict_overall["Plural"].append(plural_dict["Dative"])
      dict_overall["Plural"].append(plural_dict["Accusative"])
      dict_overall["Plural"].append(plural_dict["Ablative"])
      dict_overall["Plural"].append(plural_dict["Vocative"])
      df = pd.DataFrame(dict_overall)
      return df
    if 'm' in gender:
          dict_masc = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}

    else:
      print("not valid at the moment")
      print(gender)
      return -1


  df = make_df(decline(word))
  return df
#print(make_df(word))


# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy

      ##display(df)
      ##CREATE A FOLDER
  root = 'latin_variations_new'

  # NAN ARE REPALCED BY SPACE CHARACTER
  df.to_csv(f'{root}/{word}.csv')


  if(show):display(df)
  ## READ
  #print('READ FROM FILE ------------------------------------')
  #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
#generate_variations('Ῥωμαῖος','greek',show=True)
generate_variations('ambulo','latin',show=True)
#test words
#'Ῥωμαῖος' romans
#'μισώ' hate
#'εἰμί' is/are to be
#ελληνικά greeks
#odium hatred
#romanus
#odio hate
#απεχθάνομαι
#lacedaemon

# TO SHOW THE DATAFRAME PASS SHOW AS TRUE





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.0/845.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 kB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 13.2 MB/s eta 0:00:00
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp310-cp310-linux_x86_64.whl size=159968 sha256=7f286e2a4c8f6dcdefa2f6f43c5aa9339a3b9186bb38701553712ae9420effdd
  Stored in directory: /root/.cache/pip/wheels/7b/c3/05/60b4747cf52e0f6b6ee52022088a4de07d755016493e86373d
  Created wheel for stringcase: filename=stringcase-1.2.0-

INFO:CLTK:Cloning 'lat_models_cltk' from 'https://github.com/cltk/lat_models_cltk.git'


['lat_text_perseus', 'lat_treebank_perseus', 'lat_text_latin_library', 'phi5', 'phi7', 'latin_proper_names_cltk', 'lat_models_cltk', 'latin_pos_lemmata_cltk', 'latin_treebank_index_thomisticus', 'latin_lexica_perseus', 'latin_training_set_sentence_cltk', 'latin_word2vec_cltk', 'latin_text_antique_digiliblt', 'latin_text_corpus_grammaticorum_latinorum', 'latin_text_poeti_ditalia', 'lat_text_tesserae', 'cltk_lat_lewis_elementary_lexicon']
[('ambulo', 'v1spia---'), ('ambulas', 'v2spia---'), ('ambulat', 'v3spia---'), ('ambulamus', 'v1ppia---'), ('ambulatis', 'v2ppia---'), ('ambulant', 'v3ppia---'), ('ambulabam', 'v1siia---'), ('ambulabas', 'v2siia---'), ('ambulabat', 'v3siia---'), ('ambulabamus', 'v1piia---'), ('ambulabatis', 'v2piia---'), ('ambulabant', 'v3piia---'), ('ambulabo', 'v1sfia---'), ('ambulabis', 'v2sfia---'), ('ambulabit', 'v3sfia---'), ('ambulabimus', 'v1pfia---'), ('ambulabitis', 'v2pfia---'), ('ambulabunt', 'v3pfia---'), ('ambulavi', 'v1sria---'), ('ambulavisti', 'v2sria---

,Singular,Plural
Person,,
First Person,ambulo,ambulo
Second Person,ambulas,ambulas
Third Person,ambulat,ambulat


In [6]:


###### HELPER FUNCTIONS#######


def path_to_folder(path):
    # converts non folders into folders
    #aka just adds a slash to the end, if you dont give it a folder then it will probably produce an errror
    root = f"{path}"
    if root[-1] != "/":
        root+="/"
    return root


def get_each_file_in_folder(path):
    # this only takes in a folder path
    #RETURNS A FILE LIST
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            filelist.append(f)
    return filelist



def get_each_csv_in_folder(path):
    # takes in Noun, Adjective,Verb
    #Returns All csv's in a folder
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            if f.endswith('.csv'):
                filelist.append(f)
    return filelist

def get_each_df_in_folder(path):
    #Takes in a folder
    #Gets every dataframe in a folder
    dflist =[]
    root =path_to_folder(path)
    for f in get_each_csv_in_folder(path):
        df = pd.read_csv(f).fillna('')

        dflist.append(df)
    return dflist
'''
def get_greek_df(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)
    for f in get_each_csv_in(path):
        df = pd.read_csv(f)
        greekdf=GreekCsv(f,df)
        dflist.append(greekdf)
    return dflist
'''
# get greek words from dictionary
def get_words_from_dict(path):
    wordlist = []
    for df in get_each_df_in_folder(path):
        for i in df.index:
             wordlist.append(df.iloc[i,1])
    return wordlist

#reads all csv and intializes all words into variations
def init_variations(path):
    # takes in a folder path
    # goes though every word in a dictionary and generates the variations of that word
    for word in get_words_from_dict(path):
        #display(df.iloc[i,1])
        generate_variations(word)

def match_words_in_folder(searchpath,wordpath):
#takes in a search path and a word path it will read a word from the word path and then find all
#files in each folder that contains that word
    filelist = []
    searchpath= path_to_folder(searchpath)
    for word in get_words_from_dict(wordpath):
        if(os.path.exists(searchpath + word)):
            filelist.append(get_each_file_in_folder(searchpath + word))
    flat_list = [item for sublist in filelist for item in sublist]
    return flat_list

def get_each_df_from_path_list(filelist):
    #takes in a list of csv paths and outputs a list of dataframes
    dflist=[]
    for file in filelist:
        df = pd.read_csv(file).fillna('')
        dflist.append(df)
    return dflist

def display_list(list):
# takes in a list of dataframes and displays the data
    for i in list:
        display(i)

def display_all():
    init_variations("dictionary/")
    matchedwords= match_words_in_folder("variations/")
    display_list(get_each_df_from_path_list(matchedwords))




def get_indices(df,words):
    # returns a tuple of 2 lists that contain the row indexes and column inexes for each column and row that this word is present in
    # convert to take in a list of words
    columns=[]
    rows = []
    for columnIndex in range(df.shape[1]):
        if(isinstance(words, list)):
            for word in words:
                rowIndex = df.index[df.iloc[:, columnIndex].str.contains(word)].tolist()
                if(rowIndex):
                    columns.append(columnIndex)
                    rows.append(rowIndex)
        else:
            rowIndex = df.index[df.iloc[:, columnIndex].str.contains(words)].tolist()
            if(rowIndex):
                columns.append(columnIndex)
                rows.append(rowIndex)

            #remove duplicates

    rows = [item for sublist in rows for item in sublist]
    rows = [*set(rows)]
    #print("COLUMNINDEXES ",columns)
    #print("ROWINDEXES ",rows)
    return (rows,columns)


def get_items(dflist,row_words,column_words=''):
    # takes in a tuple of index arrays
    # empty strings are used to indicate not input standard input is either a string or array of strings
    dflist2=[]
    if (row_words=='' and column_words==''):
            print('please put a word to search in columns or rows')
            return []

    for df in dflist:#
        row_indexes = get_indices(df,row_words)[0]
        column_indexes = get_indices(df,column_words)[1]
        if (row_words=='' and column_words!=''):
            df2 =df.iloc[:,column_indexes]
        if (row_words!='' and  column_words==''):
            df2 =df.iloc[row_indexes]

        df2 =df.iloc[row_indexes,column_indexes]

        dflist2.append(df2)
    return dflist2

# a function that checks if a string contains a number
def contains_number(string):
    return any(char.isdigit() for char in string)


# not activley used will generate every possible combination kinda slow
def generate_all_accusative_scentences(verbs,nouns,sentiment):
    # takes in a verb dflist and noundf list aswell as asentiment
    sentiment = sentiment.lower()
    with open(f"latin_accusative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:
        for verbdf in verbs:
            for column_index in range(1,3):
                for row_index in range(verbdf.shape[0]):
                    verb =verbdf.iat[row_index,column_index]
                    if(verb=='' or contains_number(verb)):
                        continue
                    for noundf in nouns:
                        for column_index2 in range(1,noundf.shape[1]):
                            for row_index2 in range(noundf.shape[0]):
                                noun =noundf.iat[row_index2,column_index2]

                                if(noun== '' or contains_number(noun)):
                                    continue
                                for word in verb.split(" "):
                                    f.write(word+" O -100\n")
                                    #print(word+" O -999")
                                for i,word in enumerate(noun.split(" ")):
                                    if(sentiment =="positive" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Positive\n")
                                            #print(word +" B-ASP Positive")
                                        else:
                                            f.write(word +" I-ASP Positive\n")
                                            #print(word +" I-ASP Positive")
                                    elif(sentiment =="negative" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                        else:
                                            f.write(word +" I-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                f.write(". O -100\n\n")
                                #print(". O -999")

# WHATS MAINLY USED FOR NOW    -------------------------------------------------------------------------
def generate_random_accusative_dataset(verbs,nouns,sentiment,num):
    # takes in a verb dflist and noundf list aswell as asentiment
    # takes in number of examples to output
    #randomly selects a verb noun pair
    with open(f"latin_accusative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:
        random_verb_dfs = random.choices(verbs,k=num)
        random_noun_dfs = random.choices(nouns,k=num)
        #
        i = 0
        num_tries=0
        while i <num:
            random_verb_row = random.randrange(0,random_verb_dfs[i].shape[0])
            random_verb_column = random.randrange(1,random_verb_dfs[i].shape[1])
            random_noun_row = random.randrange(0,random_noun_dfs[i].shape[0])
            random_noun_column = random.randrange(1,random_noun_dfs[i].shape[1])

            verb =strip_accents_and_lowercase(
            random_verb_dfs[i].iat[ random_verb_row,random_verb_column])

            noun =strip_accents_and_lowercase(random_noun_dfs[i].iat[ random_noun_row,random_noun_column])
            #print(verb)
            if((noun== '' or contains_number(noun)or
            verb=='' or contains_number(verb))
            and num_tries<20 ):
                print("COULDNT FIND NUMBER OF TRIES:", num_tries)
                num_tries+=1
                continue
            num_tries = 0
            f.write("[ O -999\n")# add brackets
            for word in verb.split(" "):
                f.write(word+" O -999\n")
            for j,word in enumerate(noun.split(" ")):
                if(sentiment =="positive" ):
                    if(j==0):
                        f.write(word +" B-ASP Positive\n")
                        #print(word +" B-ASP Positive")
                    else:
                        f.write(word +" I-ASP Positive\n")
                        #print(word +" I-ASP Positive")
                elif(sentiment =="negative" ):
                    if(j==0):
                        f.write(word +" B-ASP Negative\n")
                        #print(word +" B-ASP Negative")
                    else:
                        f.write(word +" I-ASP Negative\n")
                        #print(word +" B-ASP Negative")
            f.write("] O -999\n\n")
            #randomly select item or randomly select row and column
            #check if its valid if its not randomly get another pai
            i+=1

#WHATS MAINLY USED FOR NOW
def generate_random_nominative_dataset(noun_dfs,verb_dfs,adjective_dfs,sentiment,num):
    # takes in a verb dflist and noundf list aswell as asentiment
    # takes in number of examples to output
    #randomly selects a verb noun pair
    #mathc linking plurality
    with open(f"latin_nominative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:
        random_noun_dfs = random.choices(noun_dfs,k=num)
        if(isinstance(verb_dfs, list)):
            random_verb_dfs = random.choices(verb_dfs,k=num)
        random_adjective_dfs = random.choices(adjective_dfs,k=num)
        gramatical_number=["1 sg","2 sg","3 sg","1 pl","2 pl","3 pl"]

        #
        i = 0
        num_tries=0
        while i <num:
            random_gramatical_number = random.choice(gramatical_number)


            random_noun_row = random.randrange(0,random_noun_dfs[i].shape[0])
            random_noun_column = random.randrange(1,random_noun_dfs[i].shape[1])
            #random_noun_column = get_indices(random_noun_dfs[i],random_gramatical_number)

            #random_verb_row = get_indices(random_verb_dfs[i],random_gramatical_number)
            if(isinstance(verb_dfs, list)):
                random_verb_row = random.randrange(0,random_verb_dfs[i].shape[0])
                random_verb_column = random.randrange(1,random_verb_dfs[i].shape[1])

            random_adjective_row = random.randrange(0,random_adjective_dfs[i].shape[0])
            random_adjective_column = random.randrange(1,random_adjective_dfs[i].shape[1])

            noun =strip_accents_and_lowercase(
            random_noun_dfs[i].iat[ random_noun_row,random_noun_column])

            if(isinstance(verb_dfs, list)):
                verb =random_verb_dfs[i].iat[ random_verb_row,random_verb_column]
            else:
                verb =verb_dfs

            adjective =strip_accents_and_lowercase(
            random_adjective_dfs[i].iat[ random_adjective_row,random_adjective_column])

            #print(verb)
            if((noun== '' or contains_number(noun)or
            verb=='' or contains_number(verb)or
            adjective=='' or contains_number(adjective))
            and num_tries<20 ):
                print("COULDNT FIND NUMBER OF TRIES:", num_tries)
                num_tries+=1
                continue
            num_tries = 0
            f.write("[ O -999\n")# add brackets so that stuff doesnt break when put into the
            for j,word in enumerate(noun.split(" ")):
                if(sentiment =="positive" ):
                    if(j==0):
                        f.write(word +" B-ASP Positive\n")
                        #print(word +" B-ASP Positive")
                    else:
                        f.write(word +" I-ASP Positive\n")
                        #print(word +" I-ASP Positive")
                elif(sentiment =="negative" ):
                    if(j==0):
                        f.write(word +" B-ASP Negative\n")
                        #print(word +" B-ASP Negative")
                    else:
                        f.write(word +" I-ASP Negative\n")
                        #print(word +" B-ASP Negative")
            for word in verb.split(" "):
                f.write(word+" O -999\n")
            for word in adjective.split(" "):
                f.write(word+" O -999\n")
            f.write("] O -999\n\n")
            #randomly select item or randomly select row and column
            #check if its valid if its not randomly get another pai
            i+=1
# FUNCTION GETS A RANDOM SAMPLE OUTPUTS AN ARRAY OF STUFF
def random_sample(path,num):
#READS A RANDOM SAMPLE FROM A FILE AND APPENDS IT TO AN ARRAY
    sample = []
    with open(path,"r",encoding="utf-8") as f:

        data = f.read().split('\n\n')
        for i in range (num):
            sample.append(random.choice(data))

        # read untill double new line
        # append string to array
    return sample

def sample_to_dataset(sample,name):
# ideally every entry inside of the array should be a complete dataset entry ready to be outputted directly into a file one after another
    with open(name,"+w",encoding="utf-8") as f:
        for s in(sample):
            f.write(f"{s}\n\n")

def shuffle_datasets(paths,type='test',num=2000):
#shuffle all datasets in the paths given to it and then outputs it into a new file
    combined = []
    for p in paths:
        pos =random_sample(f"{p[0]}",int(num/(2*len(paths))))
        neg =random_sample(f"{p[1]}",int(num/(2*len(paths))))
        combined += pos+neg
    random.shuffle(combined)
    sample_to_dataset(combined,f"latin.{type}.txt.atepc")




#nomitiv
#negation
#conjunction



In [7]:
#get_each_file_in("dictionary/nouns")
#get_each_csv_in("dictionary/nouns")
#init_variations_in("dictionary/nouns")
#matchedwords= match_words_in("variations/noun","dictionary/nouns")

# THIS FUNCTION PIECES TOGETHER A RANDOM SAMPLE OF DECLINED WORDS TO CREATE A ACCUSATIVE SCENTENCE
def test_create_accusative(init=False,randomize=True,debug=False):
    # INIIT VARAIATIONS CREATES THE GREEK VARIATIONS FOLDER
    if (init==True or not os.path.exists("latin_variations_new/")):
        init_variations("latin_dictionary/nouns")
        init_variations("latin_dictionary/verbs/actionverbs/negative")
        init_variations("latin_dictionary/verbs/actionverbs/positive")
        #init_variations_in("greek_dictionary/adjectives/positive")
        #init_variations_in("greek_dictionary/adjectives/negative")


    #'''
    # searches the words in the greek varations folder for nouns
    matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")

    #display_list(get_accustive_nouns(path_to_df(matchednouns)))


    matchedverbs_pos= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/actionverbs/positive")
    print(matchedverbs_pos)
    matchedverbs_neg= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/actionverbs/negative")
    print(matchedverbs_neg)
    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))

    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    verbs_pos = get_items(get_each_df_from_path_list(matchedverbs_pos),"Third Person", "Singular")
   # print(f"get_each_df_from_path_list {get_each_df_from_path_list(matchedverbs_pos)}")
    print(f"Verbs_Pos {verbs_pos}")
    verbs_neg=get_items(get_each_df_from_path_list(matchedverbs_neg),"Third Person","Singular")
   # print(f"get_each_df_from_path_list {get_each_df_from_path_list(matchedverbs_neg)}")
    print(f"Verbs_neg {verbs_neg}")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))
    nouns=get_items(get_each_df_from_path_list(matchednouns),'Accusative')
    #display(nouns)
    if debug:display_list(nouns)
    if debug:display_list(verbs_pos)
    if(randomize == True):

      print('test')
      generate_random_accusative_dataset(verbs_pos,nouns,"positive",1000)
      generate_random_accusative_dataset(verbs_neg,nouns,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")
      '''
      neg =random_sample("accusative_negative.txt.atepc",1000)
      pos=random_sample("accusative_positive.txt.atepc",1000)
      combined = neg+pos
      random.shuffle(combined)
      sample_to_dataset(combined,"greek.test.txt.atepc")


      generate_random_accusative_dataset(verbs_pos,nouns,"positive",1000)
      generate_random_accusative_dataset(verbs_neg,nouns,"negative",1000)

      neg2 =random_sample("accusative_negative.txt.atepc",1000)
      pos2=random_sample("accusative_positive.txt.atepc",1000)
      combined2 = neg2+pos2
      random.shuffle(combined2)
      sample_to_dataset(combined2,"greek.train.txt.atepc")
      '''
    else:
      generate_all_accusative_scentences(verbs_pos,nouns,"positive")
      generate_all_accusative_scentences(verbs_neg,nouns,"negative")


#---------------------------
# THIS FUNCTION PIECES TOGETHER A RANDOM SAMPLE OF DECLINED WORDS TO CREATE A NOMINATIVCE SCENTENCE
def test_create_nominative(init=False,randomize=True,debug=False):
    if (init==True or not os.path.exists("latin_variations_new/")):
        init_variations("latin_dictionary/nouns")
        init_variations("latin_dictionary/verbs/linkingverbs/")
        init_variations("latin_dictionary/adjectives/positive")
        init_variations("latin_dictionary/adjectives/negative")


    #'''

    matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")

    #display_list(get_accustive_nouns(path_to_df(matchednouns)))


    matchedverbs= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/linkingverbs/")
    matched_adj_pos= match_words_in_folder("latin_variations_new/adjective","latin_dictionary/adjectives/positive")
    matched_adj_neg= match_words_in_folder("latin_variations_new/adjective","latin_dictionary/adjectives/negative")


    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))

    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    nouns=get_items(get_each_df_from_path_list(matchednouns),'nominative')
    #verbs=get_items(path_to_df(matchedverbs),["sg","pl"],"present")
    #display_list(path_to_df(matchedverbs))
    #verbs=get_items(path_to_df(matchedverbs),"infinitive","active")
    verbs= strip_accents_and_lowercase("sum")

    adjectives_pos=get_items(get_each_df_from_path_list(matched_adj_pos),"nominative")
    adjectives_neg=get_items(get_each_df_from_path_list(matched_adj_neg),"nominative")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))

    #display(nouns)
    #if debug:display_list(nouns)
    #if debug:display_list(verbs)
    #if debug:display_list(adjectives_pos)
    if(randomize == True):
        generate_random_nominative_dataset(nouns,verbs,adjectives_pos,"positive",1000)
        generate_random_nominative_dataset(nouns,verbs,adjectives_neg,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")
        '''
        neg =random_sample("nominative_negative.txt.atepc",1000)
        pos=random_sample("nominative_positive.txt.atepc",1000)
        combined = neg+pos
        random.shuffle(combined)
        sample_to_dataset(combined,"greek.test.txt.atepc")
        '''
        '''
        generate_random_nominative_dataset(nouns,verbs,adjectives_pos,"positive",1000)
        generate_random_nominative_dataset(nouns,verbs,adjectives_neg,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")

        neg =random_sample("nominative_negative.txt.atepc",1000)
        pos=random_sample("nominative_positive.txt.atepc",1000)
        combined = neg+pos
        random.shuffle(combined)
        sample_to_dataset(combined,"greek.train.txt.atepc")
        '''

    else:
        pass
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")


In [8]:
# TEST GENERATE IS A FUNCTION THAT CREATE
def testGenerate(filename ='test', initialize = False):
    # init is a parmeter that generates all of the declension csv's
    #right now its a little innefficiant if the csv's are already initialized then you do not need to initialize so you can set to false and it will generate almost immediatley
     test_create_accusative(
     init=initialize,
     randomize=True,debug=True)

     test_create_nominative(
     init = initialize,
     randomize=True,debug=True)

     dataset_paths=[
          ["latin_accusative_positive.txt.atepc","latin_accusative_negative.txt.atepc"],
          ["latin_nominative_positive.txt.atepc","latin_nominative_negative.txt.atepc"]
          ]
     shuffle_datasets(dataset_paths,type=filename,num= 2000)

testGenerate(filename ='test',initialize = False) # set initialize to false if this is not your first time running for it to generate fast
testGenerate(filename ='train',initialize = False)

#this generates atepc files just open in a text editor download a text editor plugin for google drive to make it easier


[]
[]
Verbs_Pos []
Verbs_neg []
test


IndexError: ignored

In [ ]:
pd.read_csv("latin_variations_new/verb/diligo/_conjugation.csv")

In [ ]:
pd.read_csv("latin_variations_new/noun/aegyptus/_declension.csv")

In [ ]:
matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")
nouns=get_items(get_each_df_from_path_list(matchednouns), "accusative", "")
count = 0
for item in nouns:
  count += 1
print('\n')
for item in nouns:
  try:
    print(item["Singular"])
  except:
      print(item["1"])
      print(item["2"])
      print(item["3"])

  #print(item["Singular"])

#print(nouns[0].iloc[0])
#print(nouns[0].accusative['Singular'])
for match in lookup.match_word('casus'): # Match possible words
    print(lookup.get_dictionary_string(match))  # Start with the definition of the word
    for i in lookup.get_word_inflections(match,less=False):
        print(i) # Then print each inflection's meaning
        print()



In [ ]:
!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words
def make_df(word):
  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])


  # runs if the word has no gendered attributes
  if gender == {"-"}:
    singular_dict = {}
    plural_dict = {}
    for i in range(0,len(word)):
      # check if noun
      if word[i][1][0] == "-":
        # check if singular
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])

    dict = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict["Singular"].append(singular_dict["Nominative"])
    dict["Singular"].append(singular_dict["Genative"])
    dict["Singular"].append(singular_dict["Dative"])
    dict["Singular"].append(singular_dict["Accusative"])
    dict["Singular"].append(singular_dict["Ablative"])
    dict["Singular"].append(singular_dict["Vocative"])

    dict["Plural"].append(plural_dict["Nominative"])
    dict["Plural"].append(plural_dict["Genative"])
    dict["Plural"].append(plural_dict["Dative"])
    dict["Plural"].append(plural_dict["Accusative"])
    dict["Plural"].append(plural_dict["Ablative"])
    dict["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict)
    return df
  else:
    print("not valid at the moment")
    return -1


for item in decline("puella"):
  print(item)

word = decline("puella")
print(word)
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy
print(make_df(word))




In [ ]:
#!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words

def fill_dicts(word):
    singular_dict = {}
    plural_dict = {}
    for i in range(0, len(word)):

        # check if singular
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])
    return singular_dict, plural_dict




def make_df(word):
  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])
  print(gender)
  # runs if the word has no gendered attributes
  if gender == {"-"}:
    singular_dict, plural_dict = fill_dicts(word)
    print(singular_dict, plural_dict)
    print()
    dict = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict["Singular"].append(singular_dict["Nominative"])
    dict["Singular"].append(singular_dict["Genative"])
    dict["Singular"].append(singular_dict["Dative"])
    dict["Singular"].append(singular_dict["Accusative"])
    dict["Singular"].append(singular_dict["Ablative"])
    dict["Singular"].append(singular_dict["Vocative"])

    dict["Plural"].append(plural_dict["Nominative"])
    dict["Plural"].append(plural_dict["Genative"])
    dict["Plural"].append(plural_dict["Dative"])
    dict["Plural"].append(plural_dict["Accusative"])
    dict["Plural"].append(plural_dict["Ablative"])
    dict["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict)
    return df

  elif gender == {'f','n','m'}:
    # fills each list with the respective words
    masculine, feminine, neutral = [], [], []
    for item in word:
      if item[1][6] == 'm':
        masculine.append(item)
      elif item[1][6] == 'f':
        feminine.append(item)
      else:
        neutral.append(item)

  else:
    print("not valid at the moment")
    return -1


# for item in decline("Romanus"):
#   print(item)

word = decline("Romanus")
print(word)
print(type(word))
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy
print(make_df(word))
print('hi')



In [10]:
!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.morphology.akk import get_bound_form
from cltk.morphology.akk import decline_noun

from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

# takes in a word as a string and uses the collatinus decliner to return
# the different forms of the word. returns words which is a list that contains
# tuples formated like so: [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words

# takes in a declined word object (list of words in the format returned by the decliner function) and
# formats it so that each word is with the correct type (nominative, genative, etc.)
# currently only works with nouns that have one gender. needs to be expanded so it can make
# dictionaries for multiple genders and word types.
def fill_dicts(word, neutral = False, masc = False, fem = False):
    singular_dict = {}
    plural_dict = {}
    for i in range(0, len(word)):

      #check if verb
      if word[i][1][0] == "v":

        #check if singular verb
        if word[i][1][1:] == "1spia---":
          singular_dict["First Person"] = (word[i][0])
        if word[i][1][1:] == "2spia---":
          singular_dict["Second Person"] = (word[i][0])
        if word[i][1][1:] == "3spia---":
          singular_dict["Third Person"] = (word[i][0])
        #check if plural verb
        if word[i][1][1:] == "1ppia---":
          plural_dict["First Person"] = (word[i][0])
        if word[i][1][1:] == "2ppia---":
          plural_dict["Second Person"] = (word[i][0])
        if word[i][1][1:] == "3ppia---":
          plural_dict["Third Person"] = (word[i][0])

      # check if singular
      elif word[i][1][0] == "-":
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])
    return singular_dict, plural_dict




def make_df(word, noun = False):

  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])
  # runs if the word has no gendered attributes
  #if '-' in gender or 'n' in gender:
  singular_dict, plural_dict = fill_dicts(word, True)
  print(singular_dict, plural_dict)
  print()

  if word[1][1][0] == "v":
      dict_verb = {"Person": ["First Person", "Second Person", "Third Person"], "Singular":[], "Plural":[]}
      dict_verb["Singular"].append(singular_dict["First Person"])
      dict_verb["Singular"].append(singular_dict["Second Person"])
      dict_verb["Singular"].append(singular_dict["Third Person"])
      dict_verb["Plural"].append(singular_dict["First Person"])
      dict_verb["Plural"].append(singular_dict["Second Person"])
      dict_verb["Plural"].append(singular_dict["Third Person"])
      df = pd.DataFrame(dict_verb)
      df = df.set_index("Person")
      return df

  if word[1][1][0] == "-":
    dict_overall = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict_overall["Singular"].append(singular_dict["Nominative"])
    dict_overall["Singular"].append(singular_dict["Genative"])
    dict_overall["Singular"].append(singular_dict["Dative"])
    dict_overall["Singular"].append(singular_dict["Accusative"])
    dict_overall["Singular"].append(singular_dict["Ablative"])
    dict_overall["Singular"].append(singular_dict["Vocative"])

    dict_overall["Plural"].append(plural_dict["Nominative"])
    dict_overall["Plural"].append(plural_dict["Genative"])
    dict_overall["Plural"].append(plural_dict["Dative"])
    dict_overall["Plural"].append(plural_dict["Accusative"])
    dict_overall["Plural"].append(plural_dict["Ablative"])
    dict_overall["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict_overall)
    return df
  if 'm' in gender:
        dict_masc = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}

  else:
    print("not valid at the moment")
    print(gender)
    return -1


for item in decline("ambulo"):
  print(item)

#print(make_df(word))
word = decline("rex")
print(make_df(word))
df = make_df(word)
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy

# print('hi')



INFO:CLTK:Pulling latest 'lat_models_cltk' from 'https://github.com/cltk/lat_models_cltk.git'.


['lat_text_perseus', 'lat_treebank_perseus', 'lat_text_latin_library', 'phi5', 'phi7', 'latin_proper_names_cltk', 'lat_models_cltk', 'latin_pos_lemmata_cltk', 'latin_treebank_index_thomisticus', 'latin_lexica_perseus', 'latin_training_set_sentence_cltk', 'latin_word2vec_cltk', 'latin_text_antique_digiliblt', 'latin_text_corpus_grammaticorum_latinorum', 'latin_text_poeti_ditalia', 'lat_text_tesserae', 'cltk_lat_lewis_elementary_lexicon']
('ambulo', 'v1spia---')
('ambulas', 'v2spia---')
('ambulat', 'v3spia---')
('ambulamus', 'v1ppia---')
('ambulatis', 'v2ppia---')
('ambulant', 'v3ppia---')
('ambulabam', 'v1siia---')
('ambulabas', 'v2siia---')
('ambulabat', 'v3siia---')
('ambulabamus', 'v1piia---')
('ambulabatis', 'v2piia---')
('ambulabant', 'v3piia---')
('ambulabo', 'v1sfia---')
('ambulabis', 'v2sfia---')
('ambulabit', 'v3sfia---')
('ambulabimus', 'v1pfia---')
('ambulabitis', 'v2pfia---')
('ambulabunt', 'v3pfia---')
('ambulavi', 'v1sria---')
('ambulavisti', 'v2sria---')
('ambulavit', 'v3